<h1> Text Classification using TensorFlow/Keras on AI Platform </h1>

This notebook illustrates:
<ol>
<li> Creating datasets for AI Platform using BigQuery
<li> Creating a text classification model using the Estimator API with a Keras model
<li> Training on Cloud AI Platform
<li> Rerun with pre-trained embedding
</ol>

In [1]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [2]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-00-4f87b762258e'
PROJECT = 'qwiklabs-gcp-00-4f87b762258e'
REGION = 'us-central1'

In [3]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

if 'COLAB_GPU' in os.environ:  # this is always set on Colab, the value is 0 or 1 depending on whether a GPU is attached
  from google.colab import auth
  auth.authenticate_user()
  # download "sidecar files" since on Colab, this notebook will be on Drive
  !rm -rf txtclsmodel
  !git clone --depth 1 https://github.com/GoogleCloudPlatform/training-data-analyst
  !mv  training-data-analyst/courses/machine_learning/deepdive/09_sequence/txtclsmodel/ .
  !rm -rf training-data-analyst
  # downgrade TensorFlow to the version this notebook has been tested with
  !pip install --upgrade tensorflow==$TFVERSION

In [4]:
import tensorflow as tf
print(tf.__version__)

2.2.0-dlenv


We will look at the titles of articles and figure out whether the article came from the New York Times, TechCrunch or GitHub. 

We will use [hacker news](https://news.ycombinator.com/) as our data source. It is an aggregator that displays tech related headlines from various  sources.

### Creating Dataset from BigQuery 

Hacker news headlines are available as a BigQuery public dataset. The [dataset](https://bigquery.cloud.google.com/table/bigquery-public-data:hacker_news.stories?tab=details) contains all headlines from the sites inception in October 2006 until October 2015. 

Here is a sample of the dataset:

In [5]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [6]:
%%bigquery --project $PROJECT
SELECT
  url, title, score
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  LENGTH(title) > 10
  AND score > 10
  AND LENGTH(url) > 0
LIMIT 10

,url,title,score
0,https://www.kickstarter.com/projects/carlosxcl...,"Show HN: Code Cards, Like Texas hold 'em for p...",11
1,http://vancouver.en.craigslist.ca/van/roo/2035...,Best Roommate Ad Ever,11
2,https://github.com/Groundworkstech/Submicron,Deep-Submicron Backdoors,11
3,http://empowerunited.com/,Could this be the solution for the 99%?,11
4,http://themanufacturingrevolution.com/braun-vs...,Braun vs. Apple: Is copying designs theft or i...,11
5,https://github.com/styleguide/,"GitHub Styleguide - CSS, HTML, JS, Ruby",11
6,http://zoomzum.com/10-best-firefox-add-ons-to-...,Essential Firefox Add-Ons to Make You More Pro...,11
7,http://www.zintin.com,Feedback on our social media iPhone app,11
8,http://founderdating.com/comingtodinner/,Guess Who’s Coming to Dinner…To Save Our Company,11
9,http://tech.matchfwd.com/poor-mans-template-ab...,Poor Man's Template A/B Testing (in Django),11


Let's do some regular expression parsing in BigQuery to get the source of the newspaper article from the URL. For example, if the url is http://mobile.nytimes.com/...., I want to be left with <i>nytimes</i>

In [7]:
%%bigquery --project $PROJECT
SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  COUNT(title) AS num_articles
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
  AND LENGTH(title) > 10
GROUP BY
  source
ORDER BY num_articles DESC
LIMIT 10

,source,num_articles
0,blogspot,41386
1,github,36525
2,techcrunch,30891
3,youtube,30848
4,nytimes,28787
5,medium,18422
6,google,18235
7,wordpress,17667
8,arstechnica,13749
9,wired,12841


Now that we have good parsing of the URL to get the source, let's put together a dataset of source and titles. This will be our labeled dataset for AI Platform.

In [8]:
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT)

query="""
SELECT source, LOWER(REGEXP_REPLACE(title, '[^a-zA-Z0-9 $.-]', ' ')) AS title FROM
  (SELECT
    ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
    title
  FROM
    `bigquery-public-data.hacker_news.stories`
  WHERE
    REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
    AND LENGTH(title) > 10
  )
WHERE (source = 'github' OR source = 'nytimes' OR source = 'techcrunch')
"""

df = bq.query(query + " LIMIT 5").to_dataframe()
df.head()

,source,title
0,github,feminist-software-foundation complains about r...
1,github,expose sps as web services on the fly.
2,techcrunch,from social networks to market networks
3,github,show hn scrwl shorthand code reading and wr...
4,github,geoip module on nodejs now is a c addon


For ML training, we will need to split our dataset into training and evaluation datasets (and perhaps an independent test dataset if we are going to do model or feature selection based on the evaluation dataset).  

A simple, repeatable way to do this is to use the hash of a well-distributed column in our data (See https://www.oreilly.com/learning/repeatable-sampling-of-data-sets-in-bigquery-for-machine-learning).

In [9]:
traindf = bq.query(query + " AND ABS(MOD(FARM_FINGERPRINT(title), 4)) > 0").to_dataframe()
evaldf  = bq.query(query + " AND ABS(MOD(FARM_FINGERPRINT(title), 4)) = 0").to_dataframe()

Below we can see that roughly 75% of the data is used for training, and 25% for evaluation. 

We can also see that within each dataset, the classes are roughly balanced.

In [10]:
traindf['source'].value_counts()

github        27445
techcrunch    23131
nytimes       21586
Name: source, dtype: int64

In [11]:
evaldf['source'].value_counts()

github        9080
techcrunch    7760
nytimes       7201
Name: source, dtype: int64

Finally we will save our data, which is currently in-memory, to disk.

In [12]:
import os, shutil
DATADIR='data/txtcls'
shutil.rmtree(DATADIR, ignore_errors=True)
os.makedirs(DATADIR)
traindf.to_csv( os.path.join(DATADIR,'train.tsv'), header=False, index=False, encoding='utf-8', sep='\t')
evaldf.to_csv( os.path.join(DATADIR,'eval.tsv'), header=False, index=False, encoding='utf-8', sep='\t')

In [13]:
!head -3 data/txtcls/train.tsv

github	feminist-software-foundation complains about reddit on a github pull request
github	expose sps as web services on the fly.
github	show hn  linuxexplorer


In [14]:
!wc -l data/txtcls/*.tsv

  24041 data/txtcls/eval.tsv
  72162 data/txtcls/train.tsv
  96203 total


### TensorFlow/Keras Code

Please explore the code in this <a href="txtclsmodel/trainer">directory</a>: `model.py` contains the TensorFlow model and `task.py` parses command line arguments and launches off the training job.

In particular look for the following:

1. [tf.keras.preprocessing.text.Tokenizer.fit_on_texts()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#fit_on_texts) to generate a mapping from our word vocabulary to integers
2. [tf.keras.preprocessing.text.Tokenizer.texts_to_sequences()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences) to encode our sentences into a sequence of their respective word-integers
3. [tf.keras.preprocessing.sequence.pad_sequences()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) to pad all sequences to be the same length

The embedding layer in the keras model takes care of one-hot encoding these integers and learning a dense emedding represetation from them. 

Finally we pass the embedded text representation through a CNN model pictured below

<img src=images/txtcls_model.png  width=25%>

### Run Locally (optional step)
Let's make sure the code compiles by running locally for a fraction of an epoch.
This may not work if you don't have all the packages installed locally for gcloud (such as in Colab).
This is an optional step; move on to training on the cloud.

In [15]:
%%bash
pip install google-cloud-storage
rm -rf txtcls_trained
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=${PWD}/txtclsmodel/trainer \
   -- \
   --output_dir=${PWD}/txtcls_trained \
   --train_data_path=${PWD}/data/txtcls/train.tsv \
   --eval_data_path=${PWD}/data/txtcls/eval.tsv \
   --num_epochs=0.1

INFO:tensorflow:TF_CONFIG environment variable: {'environment': 'cloud', 'cluster': {}, 'job': {'args': ['--output_dir=/home/jupyter/training-data-analyst/courses/machine_learning/deepdive/09_sequence/txtcls_trained', '--train_data_path=/home/jupyter/training-data-analyst/courses/machine_learning/deepdive/09_sequence/data/txtcls/train.tsv', '--eval_data_path=/home/jupyter/training-data-analyst/courses/machine_learning/deepdive/09_sequence/data/txtcls/eval.tsv', '--num_epochs=0.1'], 'job_name': 'trainer.task'}, 'task': {}}
2020-07-12 13:46:18.906280: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2300000000 Hz
2020-07-12 13:46:18.906574: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5570b800c990 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-07-12 13:46:18.906620: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-07-12 13:46:18.908713: I tens

### Train on the Cloud

Let's first copy our training data to the cloud:

In [16]:
%%bash
gsutil cp data/txtcls/*.tsv gs://${BUCKET}/txtcls/

Copying file://data/txtcls/eval.tsv [Content-Type=text/tab-separated-values]...
Copying file://data/txtcls/train.tsv [Content-Type=text/tab-separated-values]...
- [2 files][  5.4 MiB/  5.4 MiB]                                                
Operation completed over 2 objects/5.4 MiB.                                      


In [17]:
%%bash
OUTDIR=gs://${BUCKET}/txtcls/trained_fromscratch
JOBNAME=txtcls_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
 --region=$REGION \
 --module-name=trainer.task \
 --package-path=${PWD}/txtclsmodel/trainer \
 --job-dir=$OUTDIR \
 --scale-tier=BASIC_GPU \
 --runtime-version 2.1 \
 --python-version 3.7 \
 -- \
 --output_dir=$OUTDIR \
 --train_data_path=gs://${BUCKET}/txtcls/train.tsv \
 --eval_data_path=gs://${BUCKET}/txtcls/eval.tsv \
 --num_epochs=5

jobId: txtcls_200712_135035
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [txtcls_200712_135035] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe txtcls_200712_135035

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs txtcls_200712_135035


Change the job name appropriately. View the job in the console, and wait until the job is complete.

In [19]:
!gcloud ai-platform jobs describe txtcls_200712_135035

createTime: '2020-07-12T13:50:38Z'
endTime: '2020-07-12T13:56:27Z'
etag: bqmw0c2l9e0=
jobId: txtcls_200712_135035
startTime: '2020-07-12T13:52:22Z'
state: SUCCEEDED
trainingInput:
  args:
  - --output_dir=gs://qwiklabs-gcp-00-4f87b762258e/txtcls/trained_fromscratch
  - --train_data_path=gs://qwiklabs-gcp-00-4f87b762258e/txtcls/train.tsv
  - --eval_data_path=gs://qwiklabs-gcp-00-4f87b762258e/txtcls/eval.tsv
  - --num_epochs=5
  jobDir: gs://qwiklabs-gcp-00-4f87b762258e/txtcls/trained_fromscratch
  packageUris:
  - gs://qwiklabs-gcp-00-4f87b762258e/txtcls/trained_fromscratch/packages/bebf57b00769f0016c3ecf3fa4aac13474a419689826ea83eab20ff33afb9b68/text_classification-1.0.tar.gz
  pythonModule: trainer.task
  pythonVersion: '3.7'
  region: us-central1
  runtimeVersion: '2.1'
  scaleTier: BASIC_GPU
trainingOutput:
  consumedMLUnits: 0.28

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/txtcls_200712_135035?project=qwiklabs-gcp-00-4f87b762258e

View logs at:

### Results
What accuracy did you get? You should see around 80%.

### Rerun with Pre-trained Embedding

We will use the popular GloVe embedding which is trained on Wikipedia as well as various news sources like the New York Times.

You can read more about Glove at the project homepage: https://nlp.stanford.edu/projects/glove/

You can download the embedding files directly from the stanford.edu site, but we've rehosted it in a GCS bucket for faster download speed.

In [20]:
!gsutil cp gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt gs://$BUCKET/txtcls/

Copying gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt [Content-Type=text/plain]...
/ [1 files][661.3 MiB/661.3 MiB]                                                
Operation completed over 1 objects/661.3 MiB.                                    


Once the embedding is downloaded re-run your cloud training job with the added command line argument: 

` --embedding_path=gs://${BUCKET}/txtcls/glove.6B.200d.txt`

While the final accuracy may not change significantly, you should notice the model is able to converge to it much more quickly because it no longer has to learn an embedding from scratch.

#### References
- This implementation is based on code from: https://github.com/google/eng-edu/tree/master/ml/guides/text_classification.
- See the full text classification tutorial at: https://developers.google.com/machine-learning/guides/text-classification/

## Next step
Client-side tokenizing in Python is hugely problematic. See <a href="text_classification_native.ipynb">Text classification with native serving</a> for how to carry out the preprocessing in the serving function itself.

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License